In [1]:
import numpy as np
from pathlib import Path
import re
import h5py


In [2]:
alle = {}
for f in Path("timeDependentVelocity/").glob("*.csv"):
    alle[float(re.match("velocity([0-3]_[0-9]{4})",f.stem)[1].replace("_","."))] = np.loadtxt(f,delimiter=",",skiprows=6)

In [3]:
tider = list(alle.keys())
tider.sort()
xy = alle[tider[0]][:,0:2]
        
for t in tider:
    alle[t] = alle[t][:,4:]
uvw = np.zeros((37,266245,3))
for i,t in enumerate(tider):
    uvw[i] = alle[t]
    
x,y = np.mgrid[0:50:3800j,0:1:76j]

In [6]:
np.asarray(tider)

array([0.0121, 0.1121, 0.2121, 0.3121, 0.4121, 0.5121, 0.6121, 0.7121,
       0.8121, 0.9121, 1.0121, 1.1121, 1.2121, 1.3121, 1.4121, 1.5121,
       1.6121, 1.7121, 1.8121, 1.9121, 2.0121, 2.1121, 2.2121, 2.3121,
       2.4121, 2.5121, 2.6121, 2.7121, 2.8121, 2.9121, 3.0121, 3.1121,
       3.2121, 3.3121, 3.4121, 3.5121, 3.6121])

In [20]:
myDPI  =150
length = 25
pts = length * 76
for i in range(37):
    resampled=griddata((xy), np.linalg.norm(uvw[i],axis=1),(x,y), method='nearest')
    fig,ax = plt.subplots(figsize=(1920/myDPI,1080/myDPI),dpi=myDPI)
    ax.imshow(resampled.T[:,:pts],extent=(0,length,0,1),origin='lower')
    # plt.show()
    fig.savefig(f"ani/chirag{i}.png")
    plt.close()

array([[    90],
       [    91],
       [    92],
       ...,
       [266242],
       [266243],
       [266244]])

In [23]:
np.argwhere(xy[:,1] ==0)

array([[     0],
       [     1],
       [    36],
       ...,
       [126886],
       [126904],
       [126922]])

In [27]:
xy[-1,0]

0.752089024

In [7]:
path = "chirag.hdf5"
print(f"Skal lagra i {path}")
with h5py.File(path, 'w') as f:
    f.create_dataset("x", data=xy, compression="gzip", compression_opts=9)
    f.create_dataset("U", data=uvw, compression="gzip", compression_opts=9)
    f.create_dataset("t", data=np.asarray(tider), compression="gzip", compression_opts=9)
    f.attrs.create('I', data=I)
    f.attrs.create('J', data=J)
    

Skal lagra i chirag.hdf5


In [ ]:
utfilnamn=Path("chirag.mp4")
slow = 1
fps = 60
t_min = tider[0]
t_max = tider[-1]
steps = (t_max - t_min) * fps


# t_list = np.arange(t_min*fps,t_max*fps)/fps
t_list = np.arange(t_min*fps,t_max*fps)/fps
t_list_part = np.arange(t_min*20,t_max*20)/20

# piv_range = ranges()
hdf5_fil = Path("chirag.hdf5")
with h5py.File(hdf5_fil, 'r') as dataset:
    (I,J)=dataset.attrs['I'],dataset.attrs['J']

    Umx = np.array(dataset['Umx'])[t_min*20:t_max*20,:]
    Umx_reshape = Umx.reshape((len(Umx),J,I))#[:,piv_range[0],piv_range[1]]
    Vmx = np.array(dataset['Vmx'])[t_min*20:t_max*20,:]
    Vmx_reshape = Vmx.reshape((len(Vmx),J,I))#[:,piv_range[0],piv_range[1]]

    # ribs = np.array(dataset['ribs'])
    
    x = np.array(dataset['x'])
    y = np.array(dataset['y'])

x_reshape = x.reshape(J,I)#[piv_range]
y_reshape = y.reshape(J,I)#[piv_range]
        
V_mag_reshape = np.hypot(Umx_reshape, Vmx_reshape)        
# V_mag_reshape = np.hypot(U[2], U[3])
V_mag_interp = interp1d(range(t_min*fps,t_max*fps,int(fps/20)), V_mag_reshape, axis=0)

myDPI = 300

fig, ax = plt.subplots(figsize=(1234/myDPI,1080/myDPI),dpi=myDPI)

field = ax.imshow(V_mag_reshape[0,:,:], extent=[x_reshape[0,0],x_reshape[0,-1], y_reshape[-1,0], y_reshape[0,0]], interpolation='none')
    
ax.set_xlim([x_reshape[0,0],x_reshape[0,-1]])
draw_rect(ax, ribs)

filmstart = [datetime.datetime.now()]

def nypkt(i):
    field.set_data(V_mag_interp(i))
    return 

#ax.axis('equal')
# ani = animation.FuncAnimation(fig, nypkt, frames=np.arange(1,steps),interval=50)
FFwriter = animation.FFMpegWriter(fps=fps)
ani = animation.FuncAnimation(fig, nypkt, frames=np.arange(0,steps-int(fps/20)),interval=(slow*1000/(fps)), blit=False)
# plt.show()

starttid = datetime.datetime.now()
ani.save(utfilnamn, writer=FFwriter)
print(f"Brukte {datetime.datetime.now()-starttid} på å lagra filmen")
plt.close()
